In [1]:
import pandas as pd
import sqlite3

In [2]:
# conectamos con la base de datos my_database.db
connection = sqlite3.connect("Cervezas_database.db")

# obtenemos un cursor que utilizamos para las queries
crsr = connection.cursor()


In [3]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

In [4]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0]) # una vez que las tablas estén creadas, deberán aparecer aquí

In [17]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

Cervezas
Bares
Empleados
Reparto


In [5]:
# hay que crear las tablas
query = '''
CREATE TABLE IF NOT EXISTS Cervezas (
    CodC int,
    Envase varchar(255),
    Capacidad int,
    Stock int,
    PRIMARY KEY(CodC)
)
'''
crsr.execute(query)

In [6]:
# hay que crear las tablas 
query = '''
CREATE TABLE IF NOT EXISTS Bares (
    CodB int,
    Nombre varchar(255),
    Cif varchar(255),
    Localidad varchar(255),
    PRIMARY KEY(CodB)
)
'''
crsr.execute(query)

In [7]:
query = '''
CREATE TABLE IF NOT EXISTS Empleados (
    CodE int,
    Nombres varchar(255),
    Sueldo varchar(255),
    PRIMARY KEY(CodE)
)
'''
crsr.execute(query)


In [8]:
query = '''
CREATE TABLE IF NOT EXISTS Reparto (
    CodE int,
    CodB int,
    CodC int,
    Fecha Date,
    Cantidad SMALLINT,
    PRIMARY KEY (CodE, CodB, CodC, Fecha), 
    FOREIGN KEY (CodE) REFERENCES Empleados(CodE),
    FOREIGN KEY (CodB) REFERENCES Bares(CodB),
    FOREIGN KEY (CodC) REFERENCES Cervezas(CodC)
);
'''
crsr.execute(query)


In [9]:
# hay que insertar los datos
query = '''
INSERT OR REPLACE INTO Cervezas VALUES ('01', 'Botella', '0,2', '3600'),
('02', 'Botella', '0,33', '1200'),
('03', 'Lata', '0,33', '2400'),
('04', 'Botella', '1', '288'),
('05', 'Barril', '60', '30')
'''
crsr.execute(query)

In [10]:
query = '''
INSERT OR REPLACE INTO Bares VALUES ('001','Stop', '11111111X','Villa Botijo'),
('002', 'Las Vegas', '22222222Y', 'Villa Botijo'),
('003', 'Club Social', NULL, 'Las Ranas'),
('004','Otra Ronda','33333333Z','La Esponja')
'''
crsr.execute(query)

In [11]:
query = '''
INSERT OR REPLACE INTO Empleados VALUES ('1','Carlos Lopez',120000),
('2', 'Rosa Perez', 110000),
('3', 'Luisa Garcia', 100000)
'''
# CRUD Create Register(insert) Update Delete
crsr.execute(query)

In [12]:
query = '''
INSERT OR REPLACE INTO Reparto VALUES ('1',  '001',  '01', '10/21/05', '240'),
('1', '001', '02', '10/21/05', '48'),
('1', '002', '03', '10/22/05', '60'),
('1', '004', '05', '10/22/05', '4'),
('2', '002', '03', '10/22/05', '48'),
('2', '002', '05', '10/23/05', '2'),
('2', '004', '01', '10/23/05', '480'),
('2', '004', '02', '10/24/05', '72'),
('3', '003', '03', '10/24/05', '48'),
('3', '003', '04', '10/25/05', '20')
'''
# CRUD Create Register(insert) Update Delete
crsr.execute(query)

In [13]:
query = '''
SELECT * FROM reparto 
'''
sql_query(query)

,CodE,CodB,CodC,Fecha,Cantidad
0,1,1,1,10/21/05,240
1,1,1,2,10/21/05,48
2,1,2,3,10/22/05,60
3,1,4,5,10/22/05,4
4,2,2,3,10/22/05,48
5,2,2,5,10/23/05,2
6,2,4,1,10/23/05,480
7,2,4,2,10/24/05,72
8,3,3,3,10/24/05,48
9,3,3,4,10/25/05,20


In [ ]:
# 1 Obtener el nombre de los empleados que hayan repartido al bar Stop durante la semana 
# del 17 al 23 de octubre de 2005.

query = '''
SELECT *
FROM Reparto
LEFT JOIN Bares
ON Reparto.CodB = Bares.CodB
LEFT JOIN Empleados
ON Reparto.CodE = Empleados.CodE
WHERE Bares.Nombre = "Stop" AND Reparto.Fecha BETWEEN '17/10/05' AND '23/10/05'
'''
sql_query(query)


,CodE,CodB,CodC,Fecha,Cantidad,CodB,Nombre,Cif,Localidad,CodE,Nombres,Sueldo


In [15]:
#2
query = '''
SELECT DISTINCT Bares.Cif
FROM Reparto
LEFT JOIN Bares
ON Reparto.CodB = Bares.CodB
LEFT JOIN Cervezas
ON Reparto.CodC = Cerveza.CodC
WHERE Cerveza.Envase = "Botella" AND Cerveza.Capacidad < 1
'''

sql_query(query)

OperationalError: no such column: Cerveza.Envase

In [16]:
#3
query = '''
SELECT DISTINCT Bares.Nombre, Empleados.Nombre, Cervezas.Envase
FROM Reparto
LEFT JOIN Bares
ON Reparto.CodB = Bares.CodB
LEFT JOIN Cervezas
ON Reparto.CodC = Cervezas.CodC
WHERE Cerveza.Envase = "Botella" AND Cerveza.Capacidad < 1
'''
sql_query(query)

OperationalError: no such column: Empleados.Nombre

In [ ]:
#4
query = '''


'''
sql_query(query)

,Nombre,Envase,Capacidad
0,Stop,Botella,0.20
1,Stop,Botella,0.33
2,Otra Ronda,Botella,0.20
3,Otra Ronda,Botella,0.33


In [ ]:
#5
query = '''
SELECT DISTINCT Bares.Nombre, Empleados.Nombre, Cervezas.Envase
FROM Reparto
LEFT JOIN Bares
ON Reparto.CodB = Bares.CodB
LEFT JOIN Cervezas
ON Reparto.CodC = Cervezas.CodC
LEFT JOIN Empleados
ON Reparto.CodE = Empleados.CodE
WHERE Bares.Nombre IN ("Stop", "Las Vegas") AND Cerveza.Envase = "Botella" 
'''
sql_query(query)

,nombreE,nombreB,Envase
0,Carlos Lopez,Stop,Botella
1,Carlos Lopez,Stop,Botella


In [ ]:
#6 
query = '''
SELECT Empleados.CodE, Empleados.Nombre, COUNT(*) as 'Viajes'
FROM Reparto
LEFT JOIN Bares
ON Reparto.CodB = Bares.CodB
LEFT JOIN Empleados
ON Reparto.CodE = Empleados.CodE
WHERE Bares.Localidad <> "villa Botijo"
GROUP By Empleados.CodE
'''
sql_query(query)

,nombreE,nombreB,Envase
0,Carlos Lopez,Stop,Botella


In [ ]:
#7
query = '''
SELECT Bares.Nombre, Bares.Localidad, SUM(Cervezas.Capacidad * Reparto.Cantidad) AS Litros
FROM Reparto
LEFT JOIN Bares
ON Reparto.CodB = Bares.CodB
LEFT JOIN Cervezas
ON Reparto.CodC = Cervezas.CodC
GROUP BY 1,2
ORDER BY 3 DESC
LIMIT 1
'''
sql_query(query)

,nombreE,nombreB,Localidad,Fecha
0,Carlos Lopez,Otra Ronda,La Esponja,2005-10-22
1,Rosa Perez,Otra Ronda,La Esponja,2005-10-23
2,Rosa Perez,Otra Ronda,La Esponja,2005-10-24
3,Luisa Garcia,Club Social,Las Ranas,2005-10-24
4,Luisa Garcia,Club Social,Las Ranas,2005-10-25


In [ ]:
#8
query = '''
SELEC DISTINC Bares.Nombre
FROM Reparto
LEFT JOIN Bares
ON Reparto.CodB = Bares.CodB
LEFT Join Cervezas
ON Reparto.CodC = Cervezas.CodC
WHERE Cervezas.Envase = "Botella" AND Cervezas.Capacidad < 1

'''
sql_query(query)

OperationalError: near "DATABASE": syntax error

In [ ]:
#9
query = '''
UPDATE Empleados
SET sueldo = sueldo * 1.05
WHERE CODE IN 
FROM (SELECT COUNT(DISTINCT Reparto.Fecha) AS 'Dias_Trabajados', Empleados.CodE, Empleados.Nombre, Empleados.Sueldo
FROM Reparto
LEFT JOIN Empleados
On Reparto.CodE = Empleados.CodE
GROUP BY Empleados.CodE
ORDER BY 1 DESC
LIMIT 1) t
)
'''
crsr.execute(query)

,CodE
0,2


In [ ]:
query = '''
SELEC *
FROM Empleados
WHERE Nombre = "Rosa Perez"
'''
sql_query(query)

In [32]:
connection.commit()
connection.close()